# Fine Tuning Flan T5

References:

1. https://www.datacamp.com/tutorial/flan-t5-tutorial
2. https://www.youtube.com/watch?v=r6XY80Z9eSA (Ignore Lightning, this also uses trainer instead of seq2seq trainer)
3. https://discuss.huggingface.co/t/trainer-vs-seq2seqtrainer/3145 (Why I'm using seq2seq trainer instead of regular trainer)

In [19]:
!nvidia-smi

Wed Jul 17 20:23:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:2B:00.0  On |                  N/A |
|  0%   32C    P8             13W /  285W |     720MiB /  16376MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
import torch
import gc

gc.collect()

if torch.cuda.is_available():
    print("CUDA is available. Device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
    torch.cuda.empty_cache() # Empty it first otherwise I get the dreaded CUDA out of memory
else:
    print("CUDA is not available.")

CUDA is available. Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Ti SUPER


In [21]:
DATASET_FILE_PATH= 'temp/zillow_qa_dataset.json'

In [22]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import nltk
import evaluate
import numpy as np
from datasets import load_dataset

dataset = load_dataset('json', data_files=DATASET_FILE_PATH)

dataset = dataset['train'].train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 561
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 141
    })
})

In [23]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    targets = [a for a in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

tokenized_dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/561 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 561
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 141
    })
})

In [24]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def extract_number(text):
    import re
    match = re.search(r'\$\d+', text)
    if match:
        return int(match.group()[1:])
    return None

In [25]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    y_true = [extract_number(label) for label in decoded_labels]
    y_pred = [extract_number(pred) for pred in decoded_preds]
    
    y_true = [val for val in y_true if val is not None]
    y_pred = [val for val in y_pred if val is not None]

    if y_true and y_pred:
        mape = mean_absolute_percentage_error(y_true, y_pred)
    else:
        mape = None

    result['mape'] = mape
    return result

In [26]:
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 6 # This was set originally to 10 for visualisation purposes

In [27]:
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   save_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   # Please note that this was left blank before - I've added this after assesing the trainer_state at checkpoint-710
   load_best_model_at_end=True,
   metric_for_best_model="eval_mape",
   greater_is_better=False if "eval_mape" else True,
)


trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)


trainer.train()


  0%|          | 0/426 [00:00<?, ?it/s]

c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.49328675866127014, 'eval_rouge1': 0.8916720395443785, 'eval_rouge2': 0.8783490937746251, 'eval_rougeL': 0.8917257683215116, 'eval_rougeLsum': 0.891697112973707, 'eval_mape': 30.780433073619704, 'eval_runtime': 28.1196, 'eval_samples_per_second': 5.014, 'eval_steps_per_second': 1.28, 'epoch': 1.0}


c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.5934332013130188, 'eval_rouge1': 0.8911383336915237, 'eval_rouge2': 0.8778171788810083, 'eval_rougeL': 0.8911454975284749, 'eval_rougeLsum': 0.8911454975284746, 'eval_mape': 25.508719430005396, 'eval_runtime': 25.6449, 'eval_samples_per_second': 5.498, 'eval_steps_per_second': 1.404, 'epoch': 6.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'train_runtime': 505.1096, 'train_samples_per_second': 6.664, 'train_steps_per_second': 0.843, 'train_loss': 0.4410615303147007, 'epoch': 6.0}


TrainOutput(global_step=426, training_loss=0.4410615303147007, metrics={'train_runtime': 505.1096, 'train_samples_per_second': 6.664, 'train_steps_per_second': 0.843, 'total_flos': 236450285457408.0, 'train_loss': 0.4410615303147007, 'epoch': 6.0})

In [28]:
model.save_pretrained('../models/t5')
tokenizer.save_pretrained('../models/t5')


('../models/t5\\tokenizer_config.json',
 '../models/t5\\special_tokens_map.json',
 '../models/t5\\spiece.model',
 '../models/t5\\added_tokens.json')